In [ ]:
'''Prepare the notebook. Run this cell first.'''
import os, re, sys
import pandas as pd
import numpy as np
from IPython.display import Image

%load_ext rpy2.ipython

In [ ]:
'''Set parameters.  Run this cell before all others.'''
study_name = '' #Change this to match your PLINK binary filestem (*.bed, *.bim, *.bam).
thousand_genomes_filestem = '~/data/1kGP/1kGP_no_ATCG_LD_pruned' #This dataset contains 36.5M LD-pruned 1kGP variants.
min_maf = 0.01 #Remove variants below this frequency. Set to 0 to keep all variants.
max_missing_sample_callrate = 0.03 #Remove samples that have greater than this percent of missing genotypes ("sample callrate").
max_missing_marker_callrate = 0.03 #Remove SNPs that have greater than this percent of missing calls ("marker callrate").
max_IBD = 0.125 #Remove samples that have identity by state greater than this coefficient of relationship (first cousins).
min_HWE_p = 0.00001 #Remove SNPs with HWE exact test p-values lower than this.
min_diffmiss_p = 0.00001 #Remove SNPs with differential missingness test p-values lower than this.

In [ ]:
'''Get the current instance's number of CPU cores and RAM.'''
ncpu_string = !nproc #Linux command to return the number of CPUs.
ncpu = int(ncpu_string[0]) #Convert to Python integer.
ram_string = !cat /proc/meminfo | fgrep MemTotal | perl -lane 'use POSIX; print floor($F[1] / 1024) #Convert from kB to MB'
ram = int(ram_string[0])

In [ ]:
'''Make analysis directory and upload your data'''

import os
study_dir = '/clink/' + study_name #Can change this variable to suit your study. 
#The base directory, /clink/, is the mountpoint for the encrypted EBS device.
if not os.path.isdir(study_dir): os.mkdir(study_dir)
%cd {study_dir}

#You can either upload data to the EC2 instance using the key pair you specified at instance launch,
#  for instance using scp from a system that has your data,
#  or you can fetch the data from this instance using SFTP/rsync/curl/wget/s3

#Remember that if you want to pull data from a private S3 bucket, the EC2 instance needs to assume an IAM role,
#  and that bucket must be configured for the EC2 instance's IAM role.

#In this example, I used scp from a firewalled and secured workstation to securely upload a Plink study to the EC2 instance.
#I used a command like this from my workstation:
#scp -i my_key_pair.pem my_study.* ec2-user@<IPv4_public_IP>:/gwas/my_study/
#  ..where <IPv4_public_IP> is the public IPv4 address of the EC2 instance running this notebook.

input_filestem  = study_name #This variable will be used throughout QC.

In [ ]:
'''Instantiate results object to hold QC results at each step.'''
class Results():
    '''Class to hold results of each QC step.'''
    def __init__(self):
        self.results = pd.DataFrame(columns=['step_number', 'step_name', 'n_snps', 'n_samples']) #Declare empty DataFrame to update.
    def add_result(self, result):
        self.results = self.results.append(result, ignore_index=True)
    def get_results(self):
        return self.results
    
class Result():
    '''Class to hold results of a single QC step.'''
    def __init__(self, step_number, step_name, n_snps_and_samples):
        self.result = pd.Series({'step_number': step_number,
                                 'step_name': step_name,
                                 'n_snps': n_snps_and_samples[0],
                                 'n_samples': n_snps_and_samples[1]})
        self.result.reindex(index=['step_number', 'step_name', 'n_snps', 'n_samples'])
    def get_result(self):
        return self.result #Simple accessor.
    
def count_snps_and_samples(filestem):
    '''Given a string representing a binary Plink filestem, count the samples and SNPs.'''
    snp_count, sample_count = 0, 0
    bim_file_name, fam_file_name = filestem + '.bim', filestem + '.fam'
    with open(bim_file_name) as bim, open(fam_file_name) as fam:
        for line in bim: snp_count += 1
        for line in fam: sample_count += 1
    return snp_count, sample_count #Tuple.
    
results = Results() #Instantiate results object to keep track of QC results at each step.
results.add_result(Result(0, 'Initial dataset', count_snps_and_samples(input_filestem)).get_result())

In [ ]:
'''1) Run sex check using plink.'''

#This should generate a *.sexcheck file for the study:
!plink --bfile $input_filestem --check-sex --out $input_filestem

In [ ]:
sexcheck_file = input_filestem + '.sexcheck'
sexprobs_file = input_filestem + '.sexprobs'

!fgrep PROBLEM $sexcheck_file > $sexprobs_file
sexproblems = !cat $sexprobs_file | wc -l
print("Number of gender discordances detected: " + str(sexproblems))

#Save the FID and IID of the individuals that failed qc, if any.
#It's okay to generate an empty file here - we'll count the lines of each fail-* file later for a sumamry.
!perl -lne 's/^\s+//; @f = split; print join "\t", (@f[0..1])' $sexprobs_file >fail-sexcheck-qc.txt

#At this point, inspect the reasons for any gender discordances.  In this case, some genders are missing in the ped file:
!head -1 $sexcheck_file #For the header.
!head -5 $sexprobs_file #Check out the first 5 discordances.

In [ ]:
'''Exclude samples that failed sexcheck QC.'''
!plink --bfile $input_filestem --remove fail-sexcheck-qc.txt --make-bed --out $input_filestem


In [ ]:
'''Update running Results() object.'''
results.add_result(Result(1, 'Sexcheck', count_snps_and_samples(input_filestem)).get_result())
results.get_results() #See the running result.

In [ ]:
'''2) Marker callrate.'''
!plink --bfile $input_filestem --geno $max_missing_marker_callrate --make-bed --out $input_filestem

In [ ]:
'''Update running Results() object.'''
results.add_result(Result(2, 'Marker callrate', count_snps_and_samples(input_filestem)).get_result())
#results.get_results() #Feel free to uncomment this to see the running result.

In [ ]:
'''3) Sample callrate.'''
!plink --bfile $input_filestem --mind $max_missing_sample_callrate --make-bed --out $input_filestem

In [ ]:
'''Update running Results() object.'''
results.add_result(Result(3, 'Sample callrate', count_snps_and_samples(input_filestem)).get_result())
#results.get_results() #Feel free to uncomment this to see the running result.

In [ ]:
'''4) MAF'''
!plink --bfile $input_filestem --maf $min_maf --make-bed --out $input_filestem

In [ ]:
'''Update running Results() object.'''
results.add_result(Result(4, 'MAF', count_snps_and_samples(input_filestem)).get_result())
#results.get_results() #Feel free to uncomment this to see the running result.

In [ ]:
'''5) IBD'''
'''Generate a missingness report and LD-pruned variants first.'''
!plink --bfile $input_filestem --missing --out $input_filestem
!plink --bfile $input_filestem --indep-pairwise 50 5 0.2 --out $input_filestem

In [ ]:
'''If the above output pruned too few or too many SNPs, adjust the linkage R^2 value 
Increase R^2 to retain more SNPs, decrease to prune more.
The goal is to end up with about 100k SNPs for a GWAS study (>= 500k SNPs genome-wide).
Then proceed with identification of related samples.'''
prune_in_file = input_filestem + '.prune.in'
unlinked_variant_count = !cat $prune_in_file | wc -l
print "Unlinked (LD-pruned) genetic variant count: " + str(unlinked_variant_count)
!plink --bfile $input_filestem --extract $prune_in_file --genome --out $input_filestem
#That command should generate a *.genome file.  This might be the most computationally heavy cell in the notebook.
#IBD calculation "used to take weeks" according to a collaborator.  Plink's IBD computation is now fast.

In [ ]:
genome_file = input_filestem + '.genome'
imiss_file = input_filestem + '.imiss'
#Clean up the whitespace:
!perl -i -lpe 's/^\s+//; s/\s+/\t/g;' $genome_file $imiss_file

#Now plot the relationships. This will take a while for a huge study!
%Rpush genome_file
%R genome <- read.table(genome_file, header=T)
%R library(ggplot2)
%R library(cowplot)
%R ggplot(genome, aes(x=Z0, y=Z1, colour=RT)) + geom_point(alpha=0.3)
%R ggsave('IBD_plot.pdf') #Save vectorized plot for presentation.
%R ggsave('IBD_plot.png') #For notebook display.
Image('IBD_plot.png')
#Inspect this plot for regions of known relationship types.  For a GWAS of presumably unrelated individuals,
#  most points should appear in the bottom right (Z0 near 1, Z1 near 0).

In [ ]:
'''These are the expected values for known types of relationships, to help you interpret this plot above:
Z0    Z1    Z2    Kinship    Relationship
0     0     1     1          Duplicated sample, monozygotic twin, or clone (future)
0     1.0   0     0.5        Parent-offspring
0.25  0.5   0.25  0.5        Full siblings
0.5   0.5   0     0.25       Half siblings
0.75  0.25  0     0.125      Cousins
1.0   0     0     0          Unrelated
'''
#This image from the R Graphical Manual (http://www.imsbio.co.jp/RGM/R_rdfile?f=GWASTools/man/ibdPlot.Rd&d=R_BC)
#  might also be helpful.
Image('http://www.imsbio.co.jp/RGM-files/R_BC/result/GWASTools/ibdPlot.Rd_001_medium.png')

In [ ]:
'''Generate a file containing FID:IID pairs of individuals to be excluded on the basis of relatedness.
Exclude the individual with greater missingness from any pair of individuals that are related more than first cousins.'''

imiss, removed = dict(), set()

with open(imiss_file) as imiss_in, open(genome_file) as genome_in, open('fail-IBD-QC.txt', 'w') as fail_IBD_out:
    #Read the missingness into a dictionary.
    header = imiss_in.readline()
    for line in imiss_in:
        fid, iid, miss_pheno, n_miss, n_geno, f_miss = line.rstrip().split("\t") #File was already whitespace-cleaned.
        imiss[(fid, iid)] = float(f_miss) #Key by FID:IID tuple, value by F_MISS (missing frequency).
        
    #Now exclude related samples. Drop whichever one has higher missingness.
    header = genome_in.readline()
    for line in genome_in:
        fid1, iid1, fid2, iid2, rt, ez, z0, z1, z2, pi_hat, phe, dst, ppc, ratio = line.rstrip().split("\t")
        if float(pi_hat) > 0.185:
            if imiss[(fid1, iid1)] >= imiss[(fid2, iid2)]: #The first sample has higher missingness.
                if (fid1, iid1) not in removed:
                    fail_IBD_out.write("\t".join([fid1, iid1]) + "\n")
                    removed.add((fid1, iid1)) #Remember that we already excluded this FID:IID pair.
            else: #The second sample has higher missingness.
                if (fid2, iid2) not in removed:
                    fail_IBD_out.write("\t".join([fid2, iid2]) + "\n")
                    removed.add((fid2, iid2))

fail_IBD_count = !cat fail-IBD-QC.txt | wc -l
print "Number of samples that failed IBD QC: " + str(fail_IBD_count)

In [ ]:
'''Now exclude the related samples from the study.'''
!plink --bfile $input_filestem --remove fail-IBD-QC.txt --make-bed --out $input_filestem

In [ ]:
'''Update running Results() object.'''
results.add_result(Result(5, 'IBD', count_snps_and_samples(input_filestem)).get_result())
#results.get_results() #Feel free to uncomment this to see the running result.

In [ ]:
'''6) Outlying heterozygosity.
First, visualize missingness and heterozygosity from reports.'''

!plink --bfile $input_filestem --missing --het --out $input_filestem
imiss_file = input_filestem + '.imiss'
het_file   = input_filestem + '.het'

#!head {imiss_file} #See how Plink generates some extra whitespace...
!perl -i -lpe 's/^\s+//; s/\s+/\t/g;' {imiss_file} {het_file} 
#!head {imiss_file} #Uncomment to see how this Perl one-liner cleaned up these two files for easy reading.

In [ ]:
'''Plot missingness versus heterozygosity. Use Rmagic.'''

%Rpush imiss_file het_file
%R imiss <- read.table(imiss_file, header=T)
%R het   <- read.table(het_file,   header=T)
%R library(ggplot2)
%R library(cowplot)

#Calculate the log10 of the F_MISS and the mean heterozygosity (among non-missing genotypes).
%R imiss$logF_MISS <- log10(imiss$F_MISS)
%R het$meanHet     <-  (het$N.NM. - het$O.HOM.)/het$N.NM.

%R -o summary_logF_MISS summary_logF_MISS <- summary(imiss$logF_MISS)
print("Summary of log10(missingness): " + str(summary_logF_MISS))

%R -o summary_meanHet summary_meanHet <- summary(het$meanHet)
print("Summary of mean heterozygosity: " + str(summary_meanHet))

%R imiss_and_het <- merge(imiss, het, by='IID')


%R imiss_vs_het_scatterplot <- ggplot(imiss_and_het, aes(x=logF_MISS, y=meanHet)) + geom_point(alpha=0.3, colour='steelblue') + xlab("Proportion of Missing Genotypes (log10)") + ylab("Mean Autosomal Heterozygosity")
%R ggsave('imiss_vs_het.pdf') #Save publication-quality PDF.
%R ggsave('imiss_vs_het.png') #Save PNG for display in the notebook.
Image('imiss_vs_het.png')
#Inspect this plot for the distribution of missingness (x-axis) versus mean heterozygosity (y-axis).
#One can exclude samples with outlying heterozygosity, e.g. more than 3 standard deviations below the mean.

#Here's something similar in pure R:
#%R plot(imiss$logF_MISS, het$meanHet, pch=1, xlab="Proportion of missing genotypes", ylab="Heterozygosity rate",axes=F); axis(2,at=c(0,0.05,0.10,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5),tick=T); axis(1,at=c(-3,-2,-1,0),labels=c(0.001,0.01,0.1,1)); abline(h=mean(het$meanHet) - (2 * sd(het$meanHet)), col="RED",  lty=2); abline(h=mean(het$meanHet) - (3 * sd(het$meanHet)), col="gray", lty=2); abline(h=mean(het$meanHet) + (2 * sd(het$meanHet)), col="red",  lty=2); abline(h=mean(het$meanHet) + (3 * sd(het$meanHet)), col="gray",  lty=2); abline(v=-1.522879, col="red", lty=2)


In [ ]:
'''Write file containing FID:IID pairs for individuals with outlying heterozygosity'''
%R het.filt <- het[which( het$meanHet > (mean(het$meanHet) + 3 * sd(het$meanHet)) | het$meanHet < (mean(het$meanHet) - 3 * sd(het$meanHet)) ),]
%R write.table(het.filt[,c(1,2)], file="fail-het-qc.txt", sep="\t", row.names=F, col.names=F, quote=F)

In [ ]:
fail_het_qc_count = !cat fail-het-qc.txt | wc -l
print("Number of individuals who failed heterozygosity QC:" + str(fail_het_qc_count))
!plink --bfile $input_filestem --remove fail-het-qc.txt --make-bed --out $input_filestem

In [ ]:
'''Update running Results() object.'''
results.add_result(Result(6, 'Outlying heterozygosity', count_snps_and_samples(input_filestem)).get_result())
#results.get_results() #Feel free to uncomment this to see the running result.

In [ ]:
'''Now that we've excluded related samples and optionally samples with outlying autosomal heterozygosity,
we repeat the marker callrate, sample callrate, and MAF steps before finally proceeding to HWE/diffmiss/PCA.'''

In [ ]:
'''7) Marker callrate.  This might not remove any SNPs, but it is important to retain guarantees about callrate.'''
!plink --bfile $input_filestem --geno $max_missing_marker_callrate --make-bed --out $input_filestem

In [ ]:
'''Update running Results() object.'''
results.add_result(Result(7, 'Marker callrate', count_snps_and_samples(input_filestem)).get_result())
#results.get_results() #Feel free to uncomment this to see the running result.

In [ ]:
'''8) Sample callrate.'''
!plink --bfile $input_filestem --mind $max_missing_sample_callrate --make-bed --out $input_filestem

In [ ]:
'''Update running Results() object.'''
results.add_result(Result(8, 'Sample callrate', count_snps_and_samples(input_filestem)).get_result())
#results.get_results() #Feel free to uncomment this to see the running result.

In [ ]:
'''9) MAF'''
!plink --bfile $input_filestem --maf $min_maf --make-bed --out $input_filestem

In [ ]:
'''Update running Results() object.'''
results.add_result(Result(9, 'MAF', count_snps_and_samples(input_filestem)).get_result())
#results.get_results() #Feel free to uncomment this to see the running result.

In [ ]:
'''10) HWE. If you wish to base this test only on cases,
which requires your PED file to have accurate case/control phenotype encodings,
you can omit the --include-nonctrl option.
The mid-p modifier is recommended by Plink.'''

!plink --bfile $input_filestem --hwe $min_HWE_p midp include-nonctrl --make-bed --out $input_filestem

In [ ]:
'''Update running Results() object.'''
results.add_result(Result(10, 'HWE', count_snps_and_samples(input_filestem)).get_result())
#results.get_results() #Feel free to uncomment this to see the running result.

In [ ]:
'''11) Differential missingness (differential sample callrate).
This step relies on accurate case/control phenotype encodings in the FAM file.'''
!plink --bfile $input_filestem --test-missing --out $input_filestem
diffmiss_file = input_filestem + '.missing'
!perl -i -lpe 's/^\s+//; s/\s+/\t/g;' {diffmiss_file} #Clean up whitespace.
diffmiss_snp_count = 0
with open('fail-diffmis-qc.txt', 'w') as fail_diffmiss, open(diffmiss_file) as diffmiss:
    header = diffmiss.readline() #Skip header line.
    for line in diffmiss:
        CHROM, SNP, F_MISS_A, F_MISS_U, P = line.rstrip().split("\t")
        if P < min_diffmiss_p:
            diffmiss_snp_count += 1
            fail_difmiss.write(SNP + "\n")

print "SNPs that failed differential missingness:", diffmiss_snp_count

In [ ]:
'''Exclude variants that failed differential missingness (if any).'''
if diffmiss_snp_count > 0:
    !plink --bfile $input_filestem --exclude diffmiss_file --make-bed --out $input_filestem
else:
    print "No SNPs show differential missingness."

In [ ]:
'''Update running Results() object.'''
results.add_result(Result(11, 'Differential missingness', count_snps_and_samples(input_filestem)).get_result())
#results.get_results() #Feel free to uncomment this to see the running result.

In [ ]:
'''12) PCA'''
#First, try to merge the study's genotypes with the 1000 Genomes Project, phase 3, version 5a.
#We'll try a 5 step approach: merge, flip, merge, exclude, merge.
merged_filestem = 'merged' #We can do this in 5 steps with 3 filestems.  This is the output filestem.
flipped_filestem = 'flipped' #This filestem holds flipped 1kGP variants that failed the first merge.
flipmerged_filestem = 'flipmerged' #This filestem holds the results of the merge with the flipped 1kGP.

In [ ]:
'''First, we'll intersect the rsIDs between the two datasets to thin down the size of the merged dataset.
Plink's --bmerge currently generates the union, which in this case will result in at least the 36.5M variants
from the 1000 Genomes Project.  Intersecting by ID might miss some rsID changes with identical chrom-pos-ref-alt.'''
snps_in_study, snps_in_1kGP = set(), set() #Populate these with IDs, then intersect them.
with open(input_filestem + '.bim') as study_bim, open(thousand_genomes_filestem + '.bim') as thousand_genomes_bim:
    for line in study_bim:
        fields = line.rstrip().split("\t")
        snps_in_study.add(fields[1]) #The second column of the BIM file holds the rsID.
    for line in thousand_genomes_bim:
        fields = line.rstrip().split("\t")
        snps_in_1kGP.add(fields[1])
        
print "SNPs in study (post-QC):", len(snps_in_study)
print "SNPs in 1000 Genomes Project:", len(snps_in_1kGP)
intersection_snps = snps_in_study.intersection(snps_in_1kGP) #Python set.
print "Intersection:", len(intersection_snps)
with open('intersection_snps.txt', 'w') as intersection_snps_file:
    for snp in intersection_snps:
        intersection_snps_file.write(snp + "\n")

In [ ]:
'''Now generate a thinned down version of each study to merge, containing only intersecting SNPs.'''
intersection_input_filestem = input_filestem + '_intersection'
intersection_1kGP_filestem  = thousand_genomes_filestem + '_intersection'
!plink --bfile $input_filestem --extract intersection_snps.txt --make-bed --out $intersection_input_filestem
!plink --bfile $thousand_genomes_filestem --extract intersection_snps.txt --make-bed --out $intersection_1kGP_filestem

In [ ]:
#1: Try an initial merge.
!plink --bfile $intersection_input_filestem --bmerge $intersection_1kGP_filestem --make-bed --out $merged_filestem

In [ ]:
'''Now, try to flip the SNPs that failed to merge.'''
missnp_file = merged_filestem + '-merge.missnp' #This holds the variants that failed to merge.
missnp_count = !wc -l $missnp_file
print "SNPs that failed to merge:", missnp_count
print "Flipping SNPs that failed to merge.\n"
!plink --bfile $intersection_1kGP_filestem --flip $missnp_file --make-bed --out $flipped_filestem

In [ ]:
'''Now try to merge with the flipped SNPs.'''
!plink --bfile $intersection_input_filestem --bmerge $flipped_filestem --make-bed --out $flipmerged_filestem

In [ ]:
'''If that above step didn't end in "Error: XXX variants with 3+ alleles present",
then you have a successful merge and can skip the next step (exclusion of flipped SNPs that failed to merge).'''
flipmerged_missnp_file = flipmerged_filestem + '-merge.missnp'
if os.path.exists(flipmerged_missnp_file): #If this file exists, this final step needs to be carried out.
    flipmerged_missnp_count = !wc -l flipmerged_missnps_file
    print "SNPs that failed to merge after flipping:", flipmerged_missnp_count
    print "Excluding flipped SNPs that failed to merge.\n"
    !plink --bfile $flipped_filestem --exclude $flipmerged_missnp_file --make-bed --out $flipped_filestem
    !plink --bfile $input_filestem --bmerge $flipped_filestem --make-bed --out $flipmerged_filestem
    #Make sure that step generated a successful merge.
else:
    print "Merge was successful: no need to exclude variants."

In [ ]:
'''Now that the study is successfully merged with 1kGP, call PC1-20 and make plots.'''
!plink --bfile $flipmerged_filestem --pca header tabs var-wts

In [ ]:
'''Now add a column that contains the population label from the 1000 Genomes project.'''
with open('plink.eigenvec') as eigenvec_in, open('/clink/1kGP/1kGP_populations.tsv') as populations_in, open('plink.eigenvec.labeled', 'w') as eigenvec_out:
    eigenvec_header = eigenvec_in.readline().rstrip() + "\tPopulation\tSuperpopulation\n" #Fix up header.
    eigenvec_out.write(eigenvec_header)
    populations = dict() #Key by sample ID (IID), value by population (26 different populations).
    populations_header = populations_in.readline() #"Sample Population\n"
    for line in populations_in:
        sample, population = line.rstrip().split("\t") #Tab delimited.
        populations[sample] = population #One of 26 populations.
    
    superpopulations = {'CHB': 'EAS', 'JPT': 'EAS', 'CHS': 'EAS', 'CDX': 'EAS', 'KHV': 'EAS',
                        'CEU': 'EUR', 'TSI': 'EUR', 'FIN': 'EUR', 'GBR': 'EUR', 'IBS': 'EUR',
                        'YRI': 'AFR', 'LWK': 'AFR', 'GWD': 'AFR', 'MSL': 'AFR', 'ESN': 'AFR', 'ASW': 'AFR', 'ACB': 'AFR',
                        'MXL': 'AMR', 'PUR': 'AMR', 'CLM': 'AMR', 'PEL': 'AMR',
                        'GIH': 'SAS', 'PJL': 'SAS', 'BEB': 'SAS', 'STU': 'SAS', 'ITU': 'SAS'}
    
    for line in eigenvec_in:
        fields = line.rstrip().split("\t") #FID, IID, PC1, PC2, ..., PC20
        IID = fields[1]
        if populations.has_key(fields[1]): #IID is in fields[1].
            population = populations.get(IID, study_name) #Assume all non-1kGP samples are in the study.
            superpopulation = superpopulations.get(population, "NA")
        else:
            population = study_name
            superpopulation = study_name
        eigenvec_out.write("\t".join(fields) + "\t" + population + "\t" + superpopulation + "\n")

In [ ]:
'''Now plot PC1 against PC2.'''
%R pca <- read.table("plink.eigenvec.labeled", sep="\t", header=T)
%R library(ggplot2)
%R library(cowplot, quietly=T)
%R pc1_vs_pc2_plot <- ggplot(pca, aes(x=PC1, y=PC2, colour=Superpopulation)) + geom_point(alpha=0.3)
%R ggsave('pc1_vs_pc2_plot.pdf')
%R ggsave('pc1_vs_pc2_plot.png') #For viewing in the notebook.
Image("pc1_vs_pc2_plot.png")

In [ ]:
'''Update running Results() object.'''
results.add_result(Result(12, 'PCA', count_snps_and_samples(input_filestem)).get_result())
#results.get_results() #Feel free to uncomment this to see the running result.

In [ ]:
'''Final step: save results table.'''
results.get_results().to_csv('GWAS_QC_results.csv', index=False)
results.get_results()

In [ ]:
'''QC is now complete!  Download this notebook, the high-resolution graphics, and the final Plink filestem.
You can download from the running instance using the following command. You just need to use the key-pair file
for the EC2 instance (substitute its file name for my_key_pair.pem)'''
!rm *~ #Clean up intermediate files.

ip_str = !wget -qO- http://instance-data/latest/meta-data/public-ipv4 #EC2 URL for getting the public IPv4 address.
ip_address = ip_str[0] #Convert to Python string.
sftp_target = 'ec2-user@' + str(ip_address) + ':' #Base SFTP server endpoint.
dataset_sftp_target = sftp_target + study_dir + '/' + input_filestem + '.*' #For the cleaned up Plink study.
pdf_sftp_target = sftp_target + study_dir + '/' + '*.pdf' #For PDF grahics generated during GWAS QC.
csv_sftp_target = sftp_target + study_dir + '/' + 'GWAS_QC_results.csv'
pca_sftp_target = sftp_target + study_dir + '/' + 'plink.eigenvec' #Contains PC1-20, keyed by FID and IID.
notebook_sftp_target = sftp_target + '/clink/CLINK/GWAS_QC.ipynb' #If you renamed the notebook,
# change this to the new notebook name.

print 'sftp -i my_key_pair.pem ' + dataset_sftp_target
print 'sftp -i my_key_pair.pem ' + notebook_sftp_target
print 'sftp -i my_key_pair.pem ' + pdf_sftp_target
print 'sftp -i my_key_pair.pem ' + csv_sftp_target
print 'sftp -i my_key_pair.pem ' + pca_sftp_target

'''Advanced users may use git to clone the notebook and S3 to store the data.
HOWEVER, note that including any credentials (e.g. git profile, AWS credentials)
on an image of the EC2 instance is a SECURITY RISK!
If you add any credentials to the EC2 instance, DO NOT TAKE AN AMI IMAGE
OR AN EBS SNAPSHOT becuase the image will contain your credentials.'''

In [ ]:
'''Now that you have downloaded the notebook, cleaned up dataset, PDF graphics, and CSV report of QC,
you can go ahead and delete the CloudFormation stack from the AWS console.'''